# Song recommender

In [1]:
import pandas as pd
import numpy as np

import spotipy
import pickle
from tqdm.auto import tqdm

import modelling
import ux
import scraping as sc
import helper_functions as hf
import data_handler as dh
import spotifyhandler as sh

from matplotlib import pyplot
from sklearn.metrics import silhouette_score

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from yellowbrick.cluster import SilhouetteVisualizer

In [2]:
# Run Marker
update_bb_top100 = False
update_wiki_artist = False
update_song_list = False
update_audio_features = False
create_words_list = False
train_kmean = False

In [3]:
# Load existing files
hot100 = pd.read_csv('data/billboard_top100.csv')
df_audiofeatures = pd.read_csv('data/audio_features.csv')
print(df_audiofeatures.shape)
df_audiofeatures.head(3)

(275081, 20)


,artists,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Wings,Mull Of Kintyre - Remastered 1993,0.350,0.297,2,-13.781,1,0.0265,0.000285,0.0000,0.123,0.423,90.603,audio_features,5yrgufPBA1PqSIOHZHrkYa,spotify:track:5yrgufPBA1PqSIOHZHrkYa,https://api.spotify.com/v1/tracks/5yrgufPBA1Pq...,https://api.spotify.com/v1/audio-analysis/5yrg...,282533,3
1,Ben Folds Five,"Selfless, Cold and Composed",0.233,0.497,11,-9.508,1,0.0322,0.361000,0.0288,0.076,0.272,180.779,audio_features,43Fd2LMelrD0HU01n4g4cw,spotify:track:43Fd2LMelrD0HU01n4g4cw,https://api.spotify.com/v1/tracks/43Fd2LMelrD0...,https://api.spotify.com/v1/audio-analysis/43Fd...,370347,3
2,Jimmy Cliff,Give And Take,0.833,0.559,2,-13.320,1,0.0688,0.087000,0.0000,0.333,0.652,127.907,audio_features,2YYtbzCV6jKwemSv6JoI8R,spotify:track:2YYtbzCV6jKwemSv6JoI8R,https://api.spotify.com/v1/tracks/2YYtbzCV6jKw...,https://api.spotify.com/v1/audio-analysis/2YYt...,133333,4


In [4]:
#sc.recommend_song()

In [5]:
if update_wiki_artist:
    artist_Series = sc.load_or_scrape('wiki_artist')['artists']
    artist_list = list(artist_Series)

In [6]:
if update_song_list:
    song_list = sh.search_songs(artist_list)
    len(song_list)

In [7]:
if create_words_list:
    print('create words list')
    words_list = hf.txt_to_list('data/speeches.txt')
    len(words_list)

In [8]:
if create_words_list:    
    song_list_update = sh.search_songs(words_list)
    len(song_list_update)


In [9]:
## concat multiple csvs and concat it to original df
if update_audio_features:
    df_complete_update = dh.concat_csvs('audio_features_update',25)
    df_complete_update.shape
    dh.update_audiofeatures_csv(df_complete_update)

### Load audiofeatures csv into df

### Train scaler on audiofeatures df

In [10]:
# first strip of df of unessessary columns
X = modelling.to_feature_df(df_audiofeatures)
X_scaled = modelling.scale(X)

### Train Model on audiofeatures df

In [11]:
k = 29
try:
    kmeans = modelling.load_kmeans(number_clusters=k)
except:
    kmeans = modelling.train_kmeans(X_scaled, number_clusters=k)

In [12]:
clusters = kmeans.predict(X_scaled)
#clusters
pd.Series(clusters).value_counts().sort_index()

0      7063
1      7881
2     10766
3      9774
4     14415
5      7055
6     10074
7     12713
8      4416
9     12720
10     6017
11    15462
12     9523
13    10743
14     3528
15     8702
16     8374
17     7277
18    13230
19    10840
20     8044
21     9799
22    14082
23    11591
24     7851
25     2975
26     9242
27     9548
28    11376
dtype: int64

### get cluster and recommend a song

In [13]:
random_song = df_audiofeatures.sample()
random_song

,artists,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
48534,Ensemble Basset,Social Democrat,0.286,0.307,9,-27.217,0,0.0582,0.94,0.903,0.11,0.227,75.779,audio_features,12hojAk8gR2Q6nHQmFvqPu,spotify:track:12hojAk8gR2Q6nHQmFvqPu,https://api.spotify.com/v1/tracks/12hojAk8gR2Q...,https://api.spotify.com/v1/audio-analysis/12ho...,113580,3


In [14]:
random_song = modelling.to_feature_df(random_song)


In [ ]:
#ux.recommend_af()

In [15]:
#ux.ask_if_correct_song('Headshrinker - Remastered',df_audiofeatures)

In [16]:
ux.recommend()

song in database
song in df audiofeatures


C:\ProgramData\Anaconda3\envs\DA_Env\lib\site-packages\sklearn\base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- analysis_url
- artists
- duration_ms
- id
- song
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


Something went wrong with getting the song audio features


'Sorry no recommendation'

In [17]:
cluster = modelling.give_cluster(random_song,kmeans)[0]
cluster

10

In [18]:
df_audiofeatures[df_audiofeatures['cluster'] == cluster].sample()

KeyError: 'cluster'

In [ ]:
df_audiofeatures

,artists,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,cluster
0,Wings,Mull Of Kintyre - Remastered 1993,0.350,0.2970,2,-13.781,1,0.0265,0.000285,0.000000,...,0.4230,90.603,audio_features,5yrgufPBA1PqSIOHZHrkYa,spotify:track:5yrgufPBA1PqSIOHZHrkYa,https://api.spotify.com/v1/tracks/5yrgufPBA1Pq...,https://api.spotify.com/v1/audio-analysis/5yrg...,282533,3,0
1,Ben Folds Five,"Selfless, Cold and Composed",0.233,0.4970,11,-9.508,1,0.0322,0.361000,0.028800,...,0.2720,180.779,audio_features,43Fd2LMelrD0HU01n4g4cw,spotify:track:43Fd2LMelrD0HU01n4g4cw,https://api.spotify.com/v1/tracks/43Fd2LMelrD0...,https://api.spotify.com/v1/audio-analysis/43Fd...,370347,3,0
2,Jimmy Cliff,Give And Take,0.833,0.5590,2,-13.320,1,0.0688,0.087000,0.000000,...,0.6520,127.907,audio_features,2YYtbzCV6jKwemSv6JoI8R,spotify:track:2YYtbzCV6jKwemSv6JoI8R,https://api.spotify.com/v1/tracks/2YYtbzCV6jKw...,https://api.spotify.com/v1/audio-analysis/2YYt...,133333,4,18
3,SHINee,SUPERSTAR,0.726,0.7960,5,-3.853,0,0.0741,0.087900,0.000000,...,0.8680,110.010,audio_features,7jiV2MnieeHtnYqTrXmQHD,spotify:track:7jiV2MnieeHtnYqTrXmQHD,https://api.spotify.com/v1/tracks/7jiV2MnieeHt...,https://api.spotify.com/v1/audio-analysis/7jiV...,174707,4,4
4,Electric Light Orchestra,So Serious,0.697,0.8660,0,-7.632,1,0.0354,0.207000,0.000000,...,0.8500,128.185,audio_features,2jo3bwXAGhVPL1Uh3dBWiB,spotify:track:2jo3bwXAGhVPL1Uh3dBWiB,https://api.spotify.com/v1/tracks/2jo3bwXAGhVP...,https://api.spotify.com/v1/audio-analysis/2jo3...,161773,4,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275076,E-17,Each Time,0.667,0.8300,11,-6.218,1,0.0309,0.077700,0.000011,...,0.6580,100.188,audio_features,3vkqxWX7KS7pvEYGqI0z6z,spotify:track:3vkqxWX7KS7pvEYGqI0z6z,https://api.spotify.com/v1/tracks/3vkqxWX7KS7p...,https://api.spotify.com/v1/audio-analysis/3vkq...,228173,4,11
275077,Cstephh,Build Regimes Freestyle,0.638,0.1960,11,-24.406,1,0.3450,0.296000,0.000000,...,0.3110,140.600,audio_features,3EeivRbcCgxHeZF10n3wpG,spotify:track:3EeivRbcCgxHeZF10n3wpG,https://api.spotify.com/v1/tracks/3EeivRbcCgxH...,https://api.spotify.com/v1/audio-analysis/3Eei...,185111,5,21
275078,tstewart,Jess,0.569,0.8640,1,-5.716,0,0.0334,0.183000,0.632000,...,0.2710,103.013,audio_features,6bQ5MVqfN49AaBWZJRjarP,spotify:track:6bQ5MVqfN49AaBWZJRjarP,https://api.spotify.com/v1/tracks/6bQ5MVqfN49A...,https://api.spotify.com/v1/audio-analysis/6bQ5...,324320,4,23
275079,CityAlight,The Goodness of Jesus,0.306,0.3560,0,-8.726,1,0.0277,0.871000,0.000000,...,0.1190,140.266,audio_features,6iL4Am7n9div8eH9kxmDYS,spotify:track:6iL4Am7n9div8eH9kxmDYS,https://api.spotify.com/v1/tracks/6iL4Am7n9div...,https://api.spotify.com/v1/audio-analysis/6iL4...,259594,4,13


In [ ]:
song = 'The End Is Anguish'
column = 'song'
df= df_audiofeatures

In [ ]:
ux.ask_if_correct_song('The End Is Anguish',df_audiofeatures)

song in database


,artists,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,cluster
158228,Ridiculon,The End Is Anguish,0.264,0.674,7,-10.47,0,0.0483,0.0216,0.742,...,0.562,67.018,audio_features,2XrQSB9XdGj2FW39MAwuBo,spotify:track:2XrQSB9XdGj2FW39MAwuBo,https://api.spotify.com/v1/tracks/2XrQSB9XdGj2...,https://api.spotify.com/v1/audio-analysis/2XrQ...,198977,4,23


In [ ]:
song = 'Jess'

In [ ]:
song in list(df_audiofeatures.name)

True

In [ ]:
df_audiofeatures[df_audiofeatures.name == song]

,artists,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,cluster
275078,tstewart,Jess,0.569,0.864,1,-5.716,0,0.0334,0.183,0.632,...,0.271,103.013,audio_features,6bQ5MVqfN49AaBWZJRjarP,spotify:track:6bQ5MVqfN49AaBWZJRjarP,https://api.spotify.com/v1/tracks/6bQ5MVqfN49A...,https://api.spotify.com/v1/audio-analysis/6bQ5...,324320,4,6


In [ ]:
type(df_audiofeatures.name)

pandas.core.series.Series

In [ ]:
song = song.lower()

In [ ]:
song

'the end is anguish'